In [1]:
import regex as re
import string

## Loading dataset

In [2]:
from datasets import load_dataset

ds = load_dataset("ai4bharat/samanantar", "hi")

Using the latest cached version of the dataset since ai4bharat/samanantar couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'hi' at C:\Users\DEBJIT\.cache\huggingface\datasets\ai4bharat___samanantar\hi\0.0.0\ead34c5b22a0354d350e7dfdd6aece7df5a48f37 (last modified on Tue Aug 19 20:08:12 2025).


## Tokenizing hte first 10000 word and storing it in a file to read from it later

In [3]:
tgt_data = ds['train'].data['src'][:10000]

with open("tokens_sample_eng.txt", "w", encoding="utf-8") as f:
    for item in tgt_data:
        for ch in str(item):
            f.write(str(ord(ch)) + " ")

## This function helps to load the file containg the tokens and store it in a variable for use

In [4]:
def load_tokens(path):
    with open(path,'r',encoding='utf-8') as f:
        for num in f.read().split():
            yield int(num)
tokens = list(load_tokens("tokens_sample_eng.txt"))

In [5]:
len(tokens)

976002

## This function Counts frequency of adjacent token pairs

In [1]:
# def get_stats(ids):
#     counts={}
#     for pair in zip(ids,ids[1:]):
#         counts[pair] = counts.get(pair,0) + 1
#     return counts

In [2]:
def pair_count(tokens):
    count={}
    for pair in zip(tokens,tokens[1:]):
        count[pair] = count.get(pair,0) + 1
    return count

p_count = pair_count(tokens)
#print(p_count)
print(sorted(((v,k) for k,v in p_count.items()), reverse=True))

[(27297, (101, 32)), (20209, (32, 116)), (17402, (115, 32)), (17068, (116, 104)), (16353, (104, 101)), (15620, (32, 97)), (14542, (105, 110)), (14324, (100, 32)), (12928, (110, 32)), (12473, (97, 110)), (12049, (101, 114)), (11827, (116, 32)), (10500, (114, 101)), (10009, (111, 110)), (9411, (32, 111)), (9146, (32, 105)), (9144, (110, 100)), (8895, (97, 116)), (8815, (114, 32)), (8541, (101, 110)), (8090, (32, 115)), (7901, (101, 115)), (7724, (97, 114)), (7651, (105, 115)), (7628, (44, 32)), (7508, (116, 101)), (7405, (101, 100)), (7312, (116, 105)), (7141, (104, 97)), (7100, (111, 114)), (7023, (32, 119)), (6826, (121, 32)), (6772, (115, 116)), (6538, (97, 108)), (6375, (32, 99)), (6240, (110, 103)), (6181, (111, 32)), (6163, (110, 116)), (5875, (105, 116)), (5799, (102, 32)), (5758, (116, 111)), (5678, (97, 115)), (5647, (32, 104)), (5556, (111, 102)), (5532, (32, 102)), (5479, (32, 112)), (5420, (118, 101)), (5351, (108, 32)), (5323, (32, 98)), (4967, (115, 101)), (4966, (109, 101)

## We then find the most occurung pair 

In [4]:
top_pair = max(p_count,key = p_count.get)
print(top_pair)

(101, 32)


## We the merge the most occuring pair and introduce a new token in place of the pair of those two tokens

In [ ]:
def merge (ids, pair, idx):
    newids = []
    i=0
    #We use while loop instead if i as we cannot change the values of i while iterating in for loop but that is not the cas fow while loop
    while i<len(ids):
        if i<len(ids)-1 and ids[i] ==pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

## Now we make the merges table of 20000 merges

In [ ]:
j=0
ids = tokens.copy()
merges={}
while j<20000:
    stats = pair_count(ids)
    #top_pair = max(stats, key=(stats.get if stats.get >1))
    filtered_stats = {pair: count for pair, count in stats.items() if count > 1}
    if not filtered_stats:
        break
    top_pair = max(filtered_stats, key=filtered_stats.get)
    idx = 256 + j
    ids = merge(ids, top_pair, idx)
    if len(ids) == 1:
        break
    print(f"Merge: {top_pair} -> {idx}")
    merges[top_pair] = idx
    print('tokens length:', len(tokens))
    print('ids length:', len(ids))
    print(f"compression ratio: {len(tokens)/len(ids):.2f}")
    j+=1

Merge: (101, 32) -> 256
tokens length: 976002
ids length: 948705
compression ratio: 1.03
Merge: (32, 116) -> 257
tokens length: 976002
ids length: 930899
compression ratio: 1.05
Merge: (115, 32) -> 258
tokens length: 976002
ids length: 915268
compression ratio: 1.07
Merge: (105, 110) -> 259
tokens length: 976002
ids length: 900726
compression ratio: 1.08
Merge: (97, 110) -> 260
tokens length: 976002
ids length: 888253
compression ratio: 1.10
Merge: (257, 104) -> 261
tokens length: 976002
ids length: 875979
compression ratio: 1.11
Merge: (101, 114) -> 262
tokens length: 976002
ids length: 863930
compression ratio: 1.13
Merge: (100, 32) -> 263
tokens length: 976002
ids length: 852305
compression ratio: 1.15
Merge: (111, 110) -> 264
tokens length: 976002
ids length: 842296
compression ratio: 1.16
Merge: (116, 32) -> 265
tokens length: 976002
ids length: 832312
compression ratio: 1.17
Merge: (101, 110) -> 266
tokens length: 976002
ids length: 823771
compression ratio: 1.18
Merge: (261, 256

In [ ]:
merges

## We then create the vocabulatry table containg the tokens as key its corresponding bytes as values , here we also include the merged tokens from hte merges table

In [8]:
vocab = {idx:bytes([idx]) for idx in range(256)}
for (p0,p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

## Here we decode the tokens to get the text using the previously created vocab table

In [9]:
def decode(ids):
    tokens = b"".join(vocab[i] for i in ids)
    return tokens.decode('utf-8', errors='replace')

## Here we encode the text by converting the whole of text to tokens nad gettingthe pair cound and then pick the pair of tokens that should be merged next, based on the BPE merge rules. After that we get the final merged tokens

In [10]:
def encode(text):
    tokens = [ord(i) for i in text]
    while len(tokens) > 1:
        stats = pair_count(tokens)
        pair  = min(stats, key =lambda x: merges.get(x,float('inf')))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

## Calculates the compression Ratio

In [11]:
def compression_ratio(text,encode):
    tokens = [ord(i) for i in text]
    bpe_tokens = encode(text)
    ratio = len(tokens)/len(bpe_tokens)
    return ratio, len(tokens), len(bpe_tokens)

## Sample texts

In [12]:
text1 = 'The summer my mother is dying of cancer, her collarbone looks like a wooden coat hanger and the soft, flowered pajamas she wears look almost empty.'
text2 = 'I sit at her feet crying, and I tell her I want to die. In one month, my mother will be dead, but she smiles widely, and she grasps my hands—Don’t worry, she says, we are going to get better. How easily I had let those words spill from my lips.'
text3=' Every day now, I rewind, swallow the words up before they can escape, stuff them back down my throat where they belong.'

In [15]:
result1,token_len1,bpe_len1 = compression_ratio(text1,encode)
result2,token_len2,bpe_len2 = compression_ratio(text2,encode)
result3,token_len3,bpe_len3 = compression_ratio(text3,encode)
print("The compression ratio for text1: ", result1)
print(f'The length of tokens of text1 is {token_len1} was reduced to {bpe_len1}')
print("The compression ratio for text2: ", result2)
print(f'The length of tokens of text1 is {token_len2} was reduced to {bpe_len2}')
print("The compression ratio for text3: ", result3)
print(f'The length of tokens of text1 is {token_len3} was reduced to {bpe_len2}')

The compression ratio for text1:  3.8684210526315788
The length of tokens of text1 is 147 was reduced to 38
The compression ratio for text2:  4.0
The length of tokens of text1 is 244 was reduced to 61
The compression ratio for text3:  4.0
The length of tokens of text1 is 120 was reduced to 61


## Here we intoduce a regex patterns so cpature hte common ocuuring combinations of letters and punctuations e.g 'll,'d,' ','  ',etc.

In [23]:
patt = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""


## Here we create an updated encoder function which smaples the given text so that it follows the regex pattern and then we get the tokens of those texts and it follows like the rest of the prev. encoder function

In [24]:
def encode_re(text):
    tokens = []

    for token in re.findall(patt, text):
        t = [ord(c) for c in token]

        while len(t) > 1:
            stats = pair_count(t)   
            pair  = min(stats, key=lambda x: merges.get(x, float('inf')))
            if pair not in merges:
                break
            idx = merges[pair]
            t = merge(t, pair, idx) 
        tokens.extend(t)

    return tokens

In [25]:
result1,token_len1,bpe_len1 = compression_ratio(text1,encode_re)
result2,token_len2,bpe_len2 = compression_ratio(text2,encode_re)
result3,token_len3,bpe_len3 = compression_ratio(text3,encode_re)
print("The compression ratio for text1: ", result1)
print(f'The length of tokens of text1 is {token_len1} was reduced to {bpe_len1}')
print("The compression ratio for text2: ", result2)
print(f'The length of tokens of text1 is {token_len2} was reduced to {bpe_len2}')
print("The compression ratio for text3: ", result3)
print(f'The length of tokens of text1 is {token_len3} was reduced to {bpe_len2}')

The compression ratio for text1:  2.0704225352112675
The length of tokens of text1 is 147 was reduced to 71
The compression ratio for text2:  1.952
The length of tokens of text1 is 244 was reduced to 125
The compression ratio for text3:  2.4
The length of tokens of text1 is 120 was reduced to 125


In [27]:
table= {(chr(a),chr(b)):val for (a,b),val in merges.items()}

In [28]:
table

{('e', ' '): 256,
 (' ', 't'): 257,
 ('s', ' '): 258,
 ('i', 'n'): 259,
 ('a', 'n'): 260,
 ('ā', 'h'): 261,
 ('e', 'r'): 262,
 ('d', ' '): 263,
 ('o', 'n'): 264,
 ('t', ' '): 265,
 ('e', 'n'): 266,
 ('ą', 'Ā'): 267,
 ('a', 'r'): 268,
 ('o', 'r'): 269,
 ('a', 't'): 270,
 (',', ' '): 271,
 ('a', 'l'): 272,
 ('y', ' '): 273,
 ('r', 'e'): 274,
 ('o', 'f'): 275,
 ('s', 't'): 276,
 ('o', ' '): 277,
 ('i', 't'): 278,
 ('o', 'u'): 279,
 ('ă', 'g'): 280,
 ('a', ' '): 281,
 ('i', 'Ĉ'): 282,
 ('e', 'ć'): 283,
 ('Ą', 'ć'): 284,
 ('i', 'c'): 285,
 ('h', 'a'): 286,
 ('ē', ' '): 287,
 ('Ć', ' '): 288,
 ('h', ' '): 289,
 ('i', 'l'): 290,
 ('i', 'Ă'): 291,
 ('e', 's'): 292,
 ('o', 'm'): 293,
 ('e', 'd'): 294,
 ('Ę', ' '): 295,
 ('h', 'Ā'): 296,
 ('i', 's'): 297,
 ('e', 'l'): 298,
 ('ă', ' '): 299,
 ('ā', 'ĕ'): 300,
 ('a', 's'): 301,
 ('Đ', ' '): 302,
 ('d', 'i'): 303,
 ('f', 'č'): 304,
 ('e', 'c'): 305,
 ('e', 'Ă'): 306,
 ('a', 'c'): 307,
 ('o', 'l'): 308,
 ('Ą', ' '): 309,
 ('a', 'm'): 310,
 ('Ě', ' '